In [5]:
import numpy as np
import pandas as pd
import json
from scipy.signal import find_peaks
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

import src.utilities as utils
from src.slide_utilities import *

import warnings
warnings.filterwarnings('ignore')

In [7]:
adata_main = sc.read_h5ad("/data/projects/robin/segmentation/xenium_outs/merged_processed_cleaned.h5ad")

Add domain generated by ```01_nichepca_all_slide.ipynb``` to ```adata_main```

In [2]:
adata_all = sc.read_h5ad("data/adata_nichepca_all_clustered.h5ad")

In [6]:
adata_all.obs

,x,y,z,cluster,n_transcripts,density,elongation,area,avg_confidence,avg_assignment_confidence,...,Patient_Sample_ID,Disease,n_genes,celltype_l1,celltype_l1_codes,celltype_l1_prob,Biopsy_ID,leiden_0.1,leiden_0.3,leiden_0.5
cell_CRfb095a77d-1,2285.790178,1164.057381,32.829188,1,79,2.0240,2.141,39.03,0.9975,0.9916,...,X40_SLE,SLE,55,MAC,8,0.999994,0011695 - X1,0,0,0
cell_CRfb095a77d-2,2298.342622,1219.741069,31.465535,4,32,1.0850,4.614,29.50,0.9771,0.9288,...,X40_SLE,SLE,17,PT,16,1.000000,0011695 - X1,0,0,0
cell_CRfb095a77d-3,2271.434052,1245.859500,32.062796,2,67,3.0590,2.990,21.90,0.9971,0.9884,...,X40_SLE,SLE,44,FIB,6,1.000000,0011695 - X1,3,2,2
cell_CRfb095a77d-5,2257.346907,1259.955827,31.273945,2,56,2.1420,9.229,26.14,0.9964,0.9707,...,X40_SLE,SLE,40,FIB,6,0.999103,0011695 - X1,3,2,2
cell_CRfb095a77d-6,2277.712022,1237.595244,35.395148,2,9,1.6980,3.283,5.30,0.9826,0.9400,...,X40_SLE,SLE,7,DTL,4,0.582253,0011695 - X1,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell_CRc71d71060-597925,2747.360264,4957.516627,24.097605,2,73,1.5480,3.617,47.15,0.9971,0.6334,...,X24_SLE,SLE,10,MAC,8,1.000000,0011216 - X2,2,3,3
cell_CRc71d71060-598041,4592.548620,1655.438877,24.254944,2,35,0.9893,1.844,35.38,0.9951,0.6206,...,X24_SLE,SLE,6,MAC,8,1.000000,0011216 - X3,2,3,3
cell_CRc71d71060-598270,8540.113826,9224.561804,19.515240,3,46,1.3870,2.667,33.16,0.9969,0.7152,...,X21_GBM,GBM,8,FIB,6,1.000000,0011216 - X9,2,3,1
cell_CRc71d71060-598508,4264.902000,2582.697217,25.121288,3,24,1.1420,5.004,21.02,0.9914,0.7517,...,X24_SLE,SLE,6,VSM/P,20,0.998473,0011216 - X3,2,3,3


In [9]:
adata_main.obs['all_slide_leiden_0.1'] = adata_all.obs['leiden_0.1']
adata_main.obs['all_slide_leiden_0.3'] = adata_all.obs['leiden_0.3']
adata_main.obs['all_slide_leiden_0.5'] = adata_all.obs['leiden_0.5']

Add domain generated by ```01_nichepca_per_slide.ipynb``` to ```adata_main```

In [12]:
slide_set = adata_main.obs['Slide_ID'].unique()
slide_set

['0011695', '0018775', '0011707', '0011287', '0011762', '0011284', '0011546', '0011216']
Categories (8, object): ['0011216', '0011284', '0011287', '0011546', '0011695', '0011707', '0011762', '0018775']

In [18]:
adata_main.obs['per_slide_leiden_0.1_1'] = '0'
adata_main.obs['per_slide_leiden_0.1_2'] = '0'
adata_main.obs['per_slide_leiden_0.2_1'] = '0'
adata_main.obs['per_slide_leiden_0.2_2'] = '0'
adata_main.obs['per_slide_leiden_0.3_1'] = '0'
adata_main.obs['per_slide_leiden_0.3_2'] = '0'
adata_main.obs['per_slide_is_glom_tmp'] = '0'

for slide in slide_set:
    adata_per_slide = sc.read_h5ad(f"data/adata_nichepca_slide_{slide}_clustered.h5ad")

    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_leiden_0.1_1'] = adata_per_slide.obs['leiden_pcah_res_0.1']
    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_leiden_0.1_2'] = adata_per_slide.obs['leiden_pca_res_0.1']
    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_leiden_0.2_1'] = adata_per_slide.obs['leiden_pcah_res_0.2']
    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_leiden_0.2_2'] = adata_per_slide.obs['leiden_pca_res_0.2']
    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_leiden_0.3_1'] = adata_per_slide.obs['leiden_pcah_res_0.3']
    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_leiden_0.3_2'] = adata_per_slide.obs['leiden_pca_res_0.3']
    adata_main.obs.loc[adata_main.obs['Slide_ID'] == slide ,'per_slide_is_glom_tmp'] = adata_per_slide.obs['is_glom']

In [19]:
adata_main.obs

,x,y,z,cluster,n_transcripts,density,elongation,area,avg_confidence,avg_assignment_confidence,...,all_slide_leiden_0.1,all_slide_leiden_0.3,all_slide_leiden_0.5,per_slide_leiden_0.1_1,per_slide_leiden_0.1_2,per_slide_leiden_0.2_1,per_slide_leiden_0.2_2,per_slide_leiden_0.3_1,per_slide_leiden_0.3_2,per_slide_is_glom_tmp
cell_CRfb095a77d-1,2285.790178,1164.057381,32.829188,1,79,2.0240,2.141,39.03,0.9975,0.9916,...,0,0,0,1,1,0,3,0,3,0
cell_CRfb095a77d-2,2298.342622,1219.741069,31.465535,4,32,1.0850,4.614,29.50,0.9771,0.9288,...,0,0,0,1,1,0,3,0,3,0
cell_CRfb095a77d-3,2271.434052,1245.859500,32.062796,2,67,3.0590,2.990,21.90,0.9971,0.9884,...,3,2,2,0,3,3,4,3,4,0
cell_CRfb095a77d-5,2257.346907,1259.955827,31.273945,2,56,2.1420,9.229,26.14,0.9964,0.9707,...,3,2,2,0,3,3,4,3,4,0
cell_CRfb095a77d-6,2277.712022,1237.595244,35.395148,2,9,1.6980,3.283,5.30,0.9826,0.9400,...,2,3,3,0,3,3,4,3,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell_CRc71d71060-597925,2747.360264,4957.516627,24.097605,2,73,1.5480,3.617,47.15,0.9971,0.6334,...,2,3,3,0,0,0,0,0,3,0
cell_CRc71d71060-598041,4592.548620,1655.438877,24.254944,2,35,0.9893,1.844,35.38,0.9951,0.6206,...,2,3,3,0,0,0,0,0,0,0
cell_CRc71d71060-598270,8540.113826,9224.561804,19.515240,3,46,1.3870,2.667,33.16,0.9969,0.7152,...,2,3,1,0,0,0,0,0,0,0
cell_CRc71d71060-598508,4264.902000,2582.697217,25.121288,3,24,1.1420,5.004,21.02,0.9914,0.7517,...,2,3,3,0,0,0,5,0,3,0


In [20]:
adata_main.write("data/adata_nichepca_slide_clustered_tmp.h5ad")
"data/adata_nichepca_slide_clustered_tmp.h5ad"

'data/adata_nichepca_slide_clustered_tmp.h5ad'

In [ ]:
epyc/Behnam/xenium-cgn/notebooks/06_domain_identification/data/adata_nichepca_slide_clustered_tmp.h5ad